In [1]:
import sys
sys.path.append( '../..' )
import os
import torch
import torch.nn as nn 
import torch.nn.functional as F
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import chamfer_distance 

from benchmark.evaluation_recon_emd import emdModule
import numpy as np
from energy import tangent_kernel

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")
import input_output 

#torch.manual_seed(0)
torch.use_deterministic_algorithms(True, warn_only=True)

In [2]:
experiment_name = 'hippo'
#experiment_name = 'cup'
#experiment_name = 'dolphin'


In [3]:
# case 1.  "Cup"
if experiment_name=='cup':
    # two different Cups 
    [VS,FS,FunS] = input_output.loadData("../../data/matching/cup2.ply")
    [VT,FT,FunT] = input_output.loadData("../../data/matching/cup3_broken.ply")
    source = [VS,FS]
    target= [VT,FT]
    # original size
    print(VS.shape)
    print(VT.shape)
    # Option 1 Sampling
    # Decimate source mesh to compute initialization for the multires algorithm 
    param_decimation = {'factor':31/32,'Vol_preser':1, 'Fun_Error_Metric': 1, 'Fun_weigth':0.00} #decimate by a factor of 16
    [verts1,faces1]= input_output.decimate_mesh(VS,FS,param_decimation)
    print(verts1.shape)
    param_decimation = {'factor':31/32,'Vol_preser':1, 'Fun_Error_Metric': 1, 'Fun_weigth':0.00} #decimate by a factor of 16
    [verts2,faces2]= input_output.decimate_mesh(VT,FT,param_decimation)
    print(verts2.shape)
    verts1 = torch.FloatTensor(verts1)
    verts2 = torch.FloatTensor(verts2)
    faces1 = torch.LongTensor(faces1)
    faces2 = torch.LongTensor(faces2)
# Case 2 Dolphin
elif experiment_name == 'dolphin':
    # sphere to dolphine 
    src_mesh = ico_sphere(4)
    VT, FT, FunS = load_obj("../../data/matching/dolphin.obj")
    VS, FS = src_mesh.verts_packed(), src_mesh.faces_packed()
    verts1, faces1 = VS, FS
    verts2, faces2 = VT, FT.verts_idx
# Case 3 Hippocampus 
elif experiment_name == 'hippo':
    verts1, faces1, verts2, faces2 = torch.load('../../data/matching/hippos_red.pt')


In [4]:
# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx1 = faces1.to(device)
faces_idx2 = faces2.to(device)

# Mark: obj files
#faces_idx1 = faces1.to(device)#.verts_idx.to(device)
#faces_idx2 = faces2.verts_idx.to(device)

verts1 = verts1.to(device)
verts2 = verts2.to(device)
# We scale normalize and center the target mesh to fit in a sphere of radius 1 centered at (0,0,0). 
# (scale, center) will be used to bring the predicted mesh to its original center and scale
# Note that normalizing the target mesh, speeds up the optimization but is not necessary!
#'''
center1 = verts1.mean(0)
center2 = verts2.mean(0)
verts1 = verts1 - center1
verts2 = verts2 - center2
scale1 = max(verts1.abs().max(0)[0])
scale2 = max(verts2.abs().max(0)[0])
verts1 = verts1 / scale1
verts2 = verts2 / scale2
#'''
# We construct a Meshes structure for the target mesh
src_mesh = Meshes(verts=[verts1], faces=[faces_idx1])
trg_mesh = Meshes(verts=[verts2], faces=[faces_idx2])
print(verts2.shape)
print(faces2.shape)

torch.Size([1654, 3])
torch.Size([3304, 3])


In [5]:
class testnet(nn.Module):
    def __init__(self):
        super(testnet, self).__init__()
        self.net = nn.Sequential(nn.Linear(6,64),nn.ReLU(),nn.Linear(64,128),nn.ReLU(),nn.Linear(128,3))
    def forward(self,x):
        return self.net(x)

In [6]:
# optimizer setting
models = testnet().cuda()
optimizer = torch.optim.Adam(models.parameters(), lr=.001)

# Number of optimization steps
Niter = 10001

# loss parameters
# weight for emd loss
w_emd = 1.0
# Plot period for the losses
plot_period = 1000

# EMD loss function
emd = emdModule()

In [7]:

best = None
best_loss = 0
best_iter = 0
for i in range(Niter):

    # Initialize optimizer
    optimizer.zero_grad()
    # model train
    sv, sf = src_mesh.get_mesh_verts_faces(0)
    sn = src_mesh.verts_normals_packed()
    inputs = torch.cat([sv.cuda(),sn.cuda()],1)
    deform_verts = models(inputs) 
    
    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)
    f1 = new_src_mesh.faces_packed()
    v1 = new_src_mesh.verts_packed()
    # We sample 5k points from the surface of each mesh 
    sample_trg,sample_trg_nor = sample_points_from_meshes(trg_mesh, 4096, return_normals=True)
    sample_src,sample_src_nor = sample_points_from_meshes(new_src_mesh, 4096, return_normals=True)

    # loss EMD 
    #print(sample_src.shape)
    emd_val, _ = emd(sample_src,sample_trg, 1e-3*5, 50)
    loss_emd = emd_val.sum()
    
    loss_chamfer, _ = chamfer_distance(sample_src, sample_trg)

    # Weighted sum of the losses
    loss = loss_emd 

    if best_loss == 0:
        best = deform_verts
        best_loss = loss.detach()
        best_iter = 0
    elif best_loss > loss.detach():
        best = deform_verts
        best_loss = loss.detach()
        best_iter = i
        #torch.save(models.state_dict(),'../../results/emd_%s_n.pth'%experiment_name)
    
    # Print the losses
    if i % plot_period==0:
        print('%d Iter: total_loss %.6f EMD_loss %.6f Chamfer_loss %.6f'% (i,loss,loss_emd, loss_chamfer))
        print('current best loss is %d: %.6f'%(best_iter,best_loss))
        
    # Optimization step
    loss.backward()
    optimizer.step()

/home/juheonlee/anaconda3/envs/jax/lib/python3.8/site-packages/pytorch3d/ops/sample_points_from_meshes.py:91: UserWarning: cumsum_cuda_kernel does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  sample_face_idxs = areas_padded.multinomial(
/home/juheonlee/anaconda3/envs/jax/lib/python3.8/site-packages/torch/autograd/__init__.py:200: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8

0 Iter: total_loss 84.126343 EMD_loss 84.126343 Chamfer_loss 0.032949
current best loss is 0: 84.126343
1000 Iter: total_loss 8.336294 EMD_loss 8.336294 Chamfer_loss 0.001856
current best loss is 103: 6.409947
2000 Iter: total_loss 6.948533 EMD_loss 6.948533 Chamfer_loss 0.001509
current best loss is 1958: 6.399203
3000 Iter: total_loss 6.386633 EMD_loss 6.386633 Chamfer_loss 0.001379
current best loss is 2912: 6.034945
4000 Iter: total_loss 6.448135 EMD_loss 6.448135 Chamfer_loss 0.001390
current best loss is 3076: 5.759430
5000 Iter: total_loss 7.138906 EMD_loss 7.138906 Chamfer_loss 0.001429
current best loss is 4406: 5.627448
6000 Iter: total_loss 5.846649 EMD_loss 5.846649 Chamfer_loss 0.001252
current best loss is 5506: 5.301846
7000 Iter: total_loss 5.738973 EMD_loss 5.738973 Chamfer_loss 0.001196
current best loss is 6949: 4.915416
8000 Iter: total_loss 5.193116 EMD_loss 5.193116 Chamfer_loss 0.001131
current best loss is 7851: 4.763547
9000 Iter: total_loss 5.493594 EMD_loss 5

In [8]:
# Fetch the verts and faces of the final predicted mesh
new_src_mesh = src_mesh.offset_verts(best)
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

# Scale normalize back to the original target size
final_verts = (final_verts) * scale2 + center2

# Store the predicted mesh using save_obj
save_obj('../../results/%s/emd_%s.obj'%(experiment_name,experiment_name), final_verts, final_faces)
print('Done!')

Done!


In [9]:
final_chamfer,_ = chamfer_distance((final_verts.unsqueeze(0).double() - center2)/scale2, verts2.unsqueeze(0).double())
print('final Chamfer distance: %.6f'%(final_chamfer.detach().cpu().numpy()))

final Chamfer distance: 0.005638
